In [1]:
import pandas as pd
import numpy as np
# myself
from config import stock_list
stock_list.remove('sh.000001')

# sz50
# sz50 = pd.read_csv('sz50_stocks.csv')
# stock_list = sz50.code.tolist()

# sh300
# sh300 = pd.read_csv('hs300_stocks.csv')
# stock_list = sh300.code.tolist()


In [2]:
ground = pd.read_csv('./data/sh.000001.csv')

ground_date = pd.to_datetime(ground['date'])
dates = ground_date.tolist()




In [3]:
datas = {}
for stock in stock_list:
    df = pd.read_csv('./data/%s.csv' % stock)
    df['date'] = pd.to_datetime(df['date'])
    data_ = pd.merge(ground_date, df, how='left', on='date')
    
    # 缺失值处理：日期对齐时会使得有些交易日的数据为空，所以需要对缺失数据进行填充
    data_.loc[:, ['volume','turn','pctChg']] = data_.loc[:, ['volume','turn','pctChg']].fillna(0)
    data_.loc[:, ['open', 'high', 'low', 'close']] = data_.loc[:, ['open', 'high', 'low', 'close']].fillna(method='pad')
    data_.loc[:, ['open', 'high', 'low', 'close']] = data_.loc[:, ['open', 'high', 'low', 'close']].fillna(0)
    data_['code'] = stock
    data_.pctChg = data_.pctChg/100
    
    datas[stock] = data_

datas

{'sh.601398':           date       code  open  high   low  close      volume      turn  \
 0   2012-07-31  sh.601398  3.95  3.96  3.66   3.75   548949616  0.209102   
 1   2012-08-31  sh.601398  3.75  3.86  3.72   3.82   590751552  0.225025   
 2   2012-09-28  sh.601398  3.83  3.85  3.60   3.75   669772347  0.255125   
 3   2012-10-31  sh.601398  3.75  3.94  3.72   3.82   508347741  0.193636   
 4   2012-11-30  sh.601398  3.83  3.93  3.80   3.88   891995913  0.339773   
 ..         ...        ...   ...   ...   ...    ...         ...       ...   
 127 2023-02-28  sh.601398  4.32  4.33  4.28   4.30  3230839665  1.198400   
 128 2023-03-31  sh.601398  4.30  4.60  4.29   4.46  6438934851  2.388000   
 129 2023-04-28  sh.601398  4.46  4.81  4.44   4.72  5212309949  1.933200   
 130 2023-05-31  sh.601398  4.70  5.47  4.68   4.83  9381763535  3.479300   
 131 2023-06-30  sh.601398  4.82  5.12  4.73   4.82  4512766004  1.674000   
 
        pctChg  
 0   -0.050633  
 1    0.018667  
 2   -0.01

## 最大收益

In [4]:
import numpy as np
from scipy.optimize import minimize

# 定义目标函数
def objective_function(weights):
    global returns 
    global cov_matrix
    returns = returns
    cov_matrix = cov_matrix
    # 计算投资组合的收益率和风险
    portfolio_return = np.dot(weights, returns)
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    
    # 返回负的投资组合收益率，以便最小化
    return -portfolio_return

# 定义约束条件
def constraint1(weights):
    # 权重之和为1
    # 权重大于0
    return np.sum(weights) - 1

def constraint2(weights):
    # 权重之和为1
    # 权重大于0
    return weights


hs300 结果如下：

In [5]:
captial = 30000000
total_earn = 0
profit = 0
defict = 0
for idx,end_date in enumerate(dates[3:]):
    total = []
    returns = []
    buy = []
    sell = []
    pct = []
    begain_date = dates[idx]
    for stock in stock_list:
        data = datas[stock]
        
        buy_price = data[data.date == end_date].open.values
        sell_price = data[data.date == end_date].close.values
        pct_chg = data[data.date == end_date].pctChg.values
        buy.append(float(buy_price))
        sell.append(float(sell_price))
        pct.append(pct_chg)

        data = data[(data.date>=begain_date) & (data.date<end_date)]
        assert_return = np.array(data['pctChg'].tolist())
        predict_price = assert_return.mean()
        
        total.append(assert_return)
        returns.append(predict_price)
    
    vix = np.vstack(total)
    cov_matrix =np.cov(vix)
    
    # 初始权重
    initial_weights = np.ones(len(stock_list)) / len(stock_list)

    # 定义优化问题
    problems_temp = []
    problems_temp.append({'type': 'eq', 'fun': constraint1})
    problems_temp.append({'type': 'ineq', 'fun': constraint2})
    problem = tuple(problems_temp)

    # 使用L-BFGS-B方法求解最小化问题
    result = minimize(objective_function, initial_weights, method='trust-constr', constraints=problem)

    # 输出结果
    weights = result.x
    amounts = np.floor(weights*captial/buy/100)*100
    earns = 0
    print('date:%s' % end_date)
    for idx2,amount in enumerate(amounts):
        if amount>=100 and amount != np.inf:
            
            earn = amount*pct[idx2]
            print('stock:%s ,earn:%f' %(stock_list[idx2],earn))
            earns+=earn
    if earns > 0:
        profit += 1
    elif earns < 0:
        defict += 1

    total_earn += earns
    captial += earns
    print('-------captial:%f' % captial)
    print('--------------%stotal_earn:%f' % (end_date,earns))        

win_rate = profit/(profit+defict)
print(total_earn)    
print(win_rate)    
    
        

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/var/folders/t_/2h9f4gns5cx_fpr94_r_7js40000gn/T/ipykernel_75378/2962852504.py:46: RuntimeWarning: divide by zero encountered in divide
  amounts = np.floor(weights*captial/buy/100)*100


date:2012-10-31 00:00:00
stock:sh.600000 ,earn:1.761518
stock:sh.600010 ,earn:31.372550
stock:sh.600011 ,earn:1.579779
stock:sh.600015 ,earn:3.980100
stock:sh.600016 ,earn:6.548673
stock:sh.600018 ,earn:6.584364
stock:sh.600019 ,earn:2.183406
stock:sh.600028 ,earn:4.006678
stock:sh.600029 ,earn:13.489736
stock:sh.600039 ,earn:4.797048
stock:sh.600050 ,earn:-11.382114
stock:sh.600061 ,earn:-1.321586
stock:sh.600089 ,earn:-5.148206
stock:sh.600115 ,earn:13.538462
stock:sh.600183 ,earn:-1.366743
stock:sh.600196 ,earn:-4.770814
stock:sh.600219 ,earn:-4.444444
stock:sh.600346 ,earn:-2.515723
stock:sh.600383 ,earn:3.379722
stock:sh.600426 ,earn:-1.349325
stock:sh.600438 ,earn:-5.328597
stock:sh.600460 ,earn:-6.419754
stock:sh.600584 ,earn:-10.917030
stock:sh.600606 ,earn:-2.678571
stock:sh.600660 ,earn:-0.284091
stock:sh.600674 ,earn:3.913043
stock:sh.600741 ,earn:-6.540084
stock:sh.600745 ,earn:4.008222
stock:sh.600760 ,earn:-1.636364
stock:sh.600795 ,earn:3.750000
stock:sh.600803 ,earn:1.6

0.020730442424242424


sz50 结果如下：

In [6]:
captial = 30000000
total_earn = 0
profit = 0
defict = 0
for idx,end_date in enumerate(dates[3:]):
    total = []
    returns = []
    buy = []
    sell = []
    pct = []
    begain_date = dates[idx]
    for stock in stock_list:
        data = datas[stock]
        
        buy_price = data[data.date == end_date].open.values
        sell_price = data[data.date == end_date].close.values
        pct_chg = data[data.date == end_date].pctChg.values
        buy.append(float(buy_price))
        sell.append(float(sell_price))
        pct.append(pct_chg)

        data = data[(data.date>=begain_date) & (data.date<end_date)]
        assert_return = np.array(data['pctChg'].tolist())
        predict_price = assert_return.mean()
        
        total.append(assert_return)
        returns.append(predict_price)
    
    vix = np.vstack(total)
    cov_matrix =np.cov(vix)
    
    # 初始权重
    initial_weights = np.ones(len(stock_list)) / len(stock_list)

    # 定义优化问题
    problems_temp = []
    problems_temp.append({'type': 'eq', 'fun': constraint1})
    problems_temp.append({'type': 'ineq', 'fun': constraint2})
    problem = tuple(problems_temp)

    # 使用L-BFGS-B方法求解最小化问题
    result = minimize(objective_function, initial_weights, method='trust-constr', constraints=problem)

    # 输出结果
    weights = result.x
    amounts = np.floor(weights*captial/buy/100)*100
    earns = 0
    print('date:%s' % end_date)
    for idx2,amount in enumerate(amounts):
        if amount>=100 and amount != np.inf:
            
            earn = amount*pct[idx2]
            print('stock:%s ,earn:%f' %(stock_list[idx2],earn))
            earns+=earn
    if earns > 0:
        profit += 1
    elif earns < 0:
        defict += 1

    total_earn += earns
    captial += earns
    print('-------captial:%f' % captial)
    print('--------------%stotal_earn:%f' % (end_date,earns))        

win_rate = profit/(profit+defict)
print(total_earn)    
print(win_rate)    
    
        

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/var/folders/t_/2h9f4gns5cx_fpr94_r_7js40000gn/T/ipykernel_14090/2962852504.py:46: RuntimeWarning: divide by zero encountered in divide
  amounts = np.floor(weights*captial/buy/100)*100


date:2012-10-31 00:00:00
stock:sh.600010 ,earn:23843.138000
stock:sh.600028 ,earn:3690.150438
stock:sh.600030 ,earn:-3900.763468
stock:sh.600031 ,earn:-1201.249938
stock:sh.600036 ,earn:-366.141930
stock:sh.600048 ,earn:1723.977640
stock:sh.600104 ,earn:-1991.875860
stock:sh.600111 ,earn:-1691.529424
stock:sh.600196 ,earn:-3368.194684
stock:sh.600276 ,earn:-230.749007
stock:sh.600309 ,earn:-593.930620
stock:sh.600346 ,earn:-1539.622476
stock:sh.600406 ,earn:0.000000
stock:sh.600436 ,earn:-473.440707
stock:sh.600438 ,earn:-4039.076526
stock:sh.600519 ,earn:20.675358
stock:sh.600585 ,earn:1961.827197
stock:sh.600690 ,earn:131.862432
stock:sh.600745 ,earn:2108.324772
stock:sh.600809 ,earn:3467.095929
stock:sh.600887 ,earn:660.985946
stock:sh.600893 ,earn:-2159.829066
stock:sh.600900 ,earn:-575.232155
stock:sh.601012 ,earn:-432.727197
stock:sh.601088 ,earn:-13.559208
stock:sh.601166 ,earn:1227.477132
stock:sh.601288 ,earn:3373.983200
stock:sh.601318 ,earn:-846.018118
stock:sh.601398 ,earn:

自选结果如下：

In [5]:
captial = 30000
total_earn = 0
profit = 0
defict = 0
for idx,end_date in enumerate(dates[5:]):
    total = []
    returns = []
    buy = []
    sell = []
    pct = []
    begain_date = dates[idx]
    for stock in stock_list:
        data = datas[stock]
        
        buy_price = data[data.date == end_date].open.values
        sell_price = data[data.date == end_date].close.values
        pct_chg = data[data.date == end_date].pctChg.values
        buy.append(float(buy_price))
        sell.append(float(sell_price))
        pct.append(pct_chg)

        data = data[(data.date>=begain_date) & (data.date<end_date)]
        assert_return = np.array(data['pctChg'].tolist())
        predict_price = assert_return.mean()
        
        total.append(assert_return)
        returns.append(predict_price)
    
    vix = np.vstack(total)
    cov_matrix =np.cov(vix)
    
    # 初始权重
    initial_weights = np.ones(len(stock_list)) / len(stock_list)

    # 定义优化问题
    problems_temp = []
    problems_temp.append({'type': 'eq', 'fun': constraint1})
    problems_temp.append({'type': 'ineq', 'fun': constraint2})
    problem = tuple(problems_temp)

    # 使用L-BFGS-B方法求解最小化问题
    result = minimize(objective_function, initial_weights, method='trust-constr', constraints=problem)

    # 输出结果
    weights = result.x
    amounts = np.floor(weights*captial/buy/100)*100
    earns = 0
    print('date:%s' % end_date)
    for idx2,amount in enumerate(amounts):
        if amount>=100 and amount != np.inf:
            
            earn = amount*pct[idx2]
            print('stock:%s ,earn:%f' %(stock_list[idx2],earn))
            earns+=earn
    if earns > 0:
        profit += 1
    elif earns < 0:
        defict += 1

    total_earn += earns
    captial += earns
    print('-------captial:%f' % captial)
    print('--------------%stotal_earn:%f' % (end_date,earns))        

win_rate = profit/(profit+defict)
print(total_earn)    
print(win_rate)    
    
        

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/var/folders/t_/2h9f4gns5cx_fpr94_r_7js40000gn/T/ipykernel_14116/2478550939.py:46: RuntimeWarning: divide by zero encountered in divide
  amounts = np.floor(weights*captial/buy/100)*100


date:2012-12-31 00:00:00
stock:sh.600010 ,earn:12.033195
stock:sh.600028 ,earn:14.950166
stock:sh.601288 ,earn:23.076924
stock:sh.601398 ,earn:6.958763
stock:sh.601668 ,earn:25.000000
stock:sh.601899 ,earn:6.388889
-------captial:30088.407937
--------------2012-12-31 00:00:00total_earn:88.407937
date:2013-01-31 00:00:00
stock:sh.601633 ,earn:208.101264
-------captial:30296.509201
--------------2013-01-31 00:00:00total_earn:208.101264
date:2013-02-28 00:00:00
stock:sh.601288 ,earn:-5.194806
stock:sh.601398 ,earn:-4.318182
stock:sh.601668 ,earn:6.575342
stock:sh.601669 ,earn:1.837270
stock:sh.601899 ,earn:-4.392765
stock:sh.601919 ,earn:-1.877934
-------captial:30289.138126
--------------2013-02-28 00:00:00total_earn:-7.371075
date:2013-03-29 00:00:00
stock:sh.601633 ,earn:77.331560
-------captial:30366.469686
--------------2013-03-29 00:00:00total_earn:77.331560
date:2013-04-26 00:00:00
stock:sh.601633 ,earn:95.101668
-------captial:30461.571354
--------------2013-04-26 00:00:00total_ea

## 最小风险

In [4]:
from scipy.optimize import minimize
def portfolio_performance(weights, returns, cov_matrix):
    returns = np.sum(returns*weights ) *12
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(12)
    return std, returns


def minimize_volatility(weights, returns, cov_matrix):
    return portfolio_performance(weights, returns, cov_matrix)[0]



hs300 结果如下：

In [5]:
captial = 30000000
total_earn = 0
profit = 0
defict = 0
for idx,end_date in enumerate(dates[12:]):
    total = []
    returns = []
    buy = []
    sell = []
    pct = []
    begain_date = dates[idx]
    for stock in stock_list:
        data = datas[stock]
        
        buy_price = data[data.date == end_date].open.values
        sell_price = data[data.date == end_date].close.values
        pct_chg = data[data.date == end_date].pctChg.values
        buy.append(float(buy_price))
        sell.append(float(sell_price))
        pct.append(pct_chg)

        data = data[(data.date>=begain_date) & (data.date<end_date)]
        assert_return = np.array(data['pctChg'].tolist())
        predict_price = assert_return.mean()
        
        total.append(assert_return)
        returns.append(predict_price)
    
    vix = np.vstack(total)
    cov_matrix =np.cov(vix)
    
    num_assets = len(returns)
    args = (returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    initial_guess = num_assets*[1./num_assets,]

    optimal_volatility = minimize(minimize_volatility, initial_guess, args=args, bounds=bounds, constraints=constraints)
    
    
    # 输出结果
    weights = optimal_volatility['x']
    amounts = np.floor(weights*captial/buy/100)*100
    earns = 0
    print('date:%s' % end_date)
    for idx2,amount in enumerate(amounts):
        if amount>=100 and amount != np.inf:
            
            earn = amount*pct[idx2]
            print('stock:%s ,earn:%f' %(stock_list[idx2],earn))
            earns+=earn
    if earns > 0:
        profit += 1
    elif earns < 0:
        defict += 1

    total_earn += earns
    captial += earns
    print('-------captial:%f' % captial)
    print('--------------%stotal_earn:%f' % (end_date,earns))        
    
    
win_rate = profit/(profit+defict)
print(total_earn)    
print(win_rate)    

/var/folders/t_/2h9f4gns5cx_fpr94_r_7js40000gn/T/ipykernel_15473/3095519782.py:43: RuntimeWarning: divide by zero encountered in divide
  amounts = np.floor(weights*captial/buy/100)*100


date:2013-07-31 00:00:00
-------captial:30000000.000000
--------------2013-07-31 00:00:00total_earn:0.000000
date:2013-08-30 00:00:00
-------captial:30000000.000000
--------------2013-08-30 00:00:00total_earn:0.000000
date:2013-09-30 00:00:00
stock:sz.000333 ,earn:-207.360860
-------captial:29999792.639140
--------------2013-09-30 00:00:00total_earn:-207.360860
date:2013-10-31 00:00:00
stock:sz.000333 ,earn:35.800185
stock:sz.002555 ,earn:229.084042
-------captial:30000057.523367
--------------2013-10-31 00:00:00total_earn:264.884227
date:2013-11-29 00:00:00
stock:sh.600547 ,earn:-7.976654
stock:sh.601186 ,earn:27.450980
stock:sh.601398 ,earn:-2.094240
stock:sz.000333 ,earn:-2.024794
-------captial:30000072.878659
--------------2013-11-29 00:00:00total_earn:15.355292
date:2013-12-31 00:00:00
-------captial:30000072.878659
--------------2013-12-31 00:00:00total_earn:0.000000
date:2014-01-30 00:00:00
stock:sh.600176 ,earn:-12.945838
stock:sh.601225 ,earn:450.000000
stock:sh.601398 ,earn:

sz50 结果如下：

In [5]:
captial = 30000000
total_earn = 0
profit = 0
defict = 0
for idx,end_date in enumerate(dates[12:]):
    total = []
    returns = []
    buy = []
    sell = []
    pct = []
    begain_date = dates[idx]
    for stock in stock_list:
        data = datas[stock]
        
        buy_price = data[data.date == end_date].open.values
        sell_price = data[data.date == end_date].close.values
        pct_chg = data[data.date == end_date].pctChg.values
        buy.append(float(buy_price))
        sell.append(float(sell_price))
        pct.append(pct_chg)

        data = data[(data.date>=begain_date) & (data.date<end_date)]
        assert_return = np.array(data['pctChg'].tolist())
        predict_price = assert_return.mean()
        
        total.append(assert_return)
        returns.append(predict_price)
    
    vix = np.vstack(total)
    cov_matrix =np.cov(vix)
    
    num_assets = len(returns)
    args = (returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    initial_guess = num_assets*[1./num_assets,]

    optimal_volatility = minimize(minimize_volatility, initial_guess, args=args, bounds=bounds, constraints=constraints)
    
    
    # 输出结果
    weights = optimal_volatility['x']
    amounts = np.floor(weights*captial/buy/100)*100
    earns = 0
    print('date:%s' % end_date)
    for idx2,amount in enumerate(amounts):
        if amount>=100 and amount != np.inf:
            
            earn = amount*pct[idx2]
            print('stock:%s ,earn:%f' %(stock_list[idx2],earn))
            earns+=earn
    if earns > 0:
        profit += 1
    elif earns < 0:
        defict += 1

    total_earn += earns
    captial += earns
    print('-------captial:%f' % captial)
    print('--------------%stotal_earn:%f' % (end_date,earns))        
    
    
win_rate = profit/(profit+defict)
print(total_earn)    
print(win_rate)    

/var/folders/t_/2h9f4gns5cx_fpr94_r_7js40000gn/T/ipykernel_16188/3095519782.py:43: RuntimeWarning: divide by zero encountered in divide
  amounts = np.floor(weights*captial/buy/100)*100


date:2013-07-31 00:00:00
-------captial:30000000.000000
--------------2013-07-31 00:00:00total_earn:0.000000
date:2013-08-30 00:00:00
-------captial:30000000.000000
--------------2013-08-30 00:00:00total_earn:0.000000
date:2013-09-30 00:00:00
-------captial:30000000.000000
--------------2013-09-30 00:00:00total_earn:0.000000
date:2013-10-31 00:00:00
-------captial:30000000.000000
--------------2013-10-31 00:00:00total_earn:0.000000
date:2013-11-29 00:00:00
-------captial:30000000.000000
--------------2013-11-29 00:00:00total_earn:0.000000
date:2013-12-31 00:00:00
-------captial:30000000.000000
--------------2013-12-31 00:00:00total_earn:0.000000
date:2014-01-30 00:00:00
stock:sh.601225 ,earn:4260.750000
-------captial:30004260.750000
--------------2014-01-30 00:00:00total_earn:4260.750000
date:2014-02-28 00:00:00
stock:sh.603288 ,earn:18054.634277
-------captial:30022315.384277
--------------2014-02-28 00:00:00total_earn:18054.634277
date:2014-03-31 00:00:00
-------captial:30022315.384

自选结果如下：

In [5]:
captial = 30000000
total_earn = 0
profit = 0
defict = 0
for idx,end_date in enumerate(dates[12:]):
    total = []
    returns = []
    buy = []
    sell = []
    pct = []
    begain_date = dates[idx]
    for stock in stock_list:
        data = datas[stock]
        
        buy_price = data[data.date == end_date].open.values
        sell_price = data[data.date == end_date].close.values
        pct_chg = data[data.date == end_date].pctChg.values
        buy.append(float(buy_price))
        sell.append(float(sell_price))
        pct.append(pct_chg)

        data = data[(data.date>=begain_date) & (data.date<end_date)]
        assert_return = np.array(data['pctChg'].tolist())
        predict_price = assert_return.mean()
        
        total.append(assert_return)
        returns.append(predict_price)
    
    vix = np.vstack(total)
    cov_matrix =np.cov(vix)
    
    num_assets = len(returns)
    args = (returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    initial_guess = num_assets*[1./num_assets,]

    optimal_volatility = minimize(minimize_volatility, initial_guess, args=args, bounds=bounds, constraints=constraints)
    
    
    # 输出结果
    weights = optimal_volatility['x']
    amounts = np.floor(weights*captial/buy/100)*100
    earns = 0
    print('date:%s' % end_date)
    for idx2,amount in enumerate(amounts):
        if amount>=100 and amount != np.inf:
            
            earn = amount*pct[idx2]
            print('stock:%s ,earn:%f' %(stock_list[idx2],earn))
            earns+=earn
    if earns > 0:
        profit += 1
    elif earns < 0:
        defict += 1

    total_earn += earns
    captial += earns
    print('-------captial:%f' % captial)
    print('--------------%stotal_earn:%f' % (end_date,earns))        
    
    
win_rate = profit/(profit+defict)
print(total_earn)    
print(win_rate)    

/var/folders/t_/2h9f4gns5cx_fpr94_r_7js40000gn/T/ipykernel_16258/3095519782.py:43: RuntimeWarning: divide by zero encountered in divide
  amounts = np.floor(weights*captial/buy/100)*100


date:2013-07-31 00:00:00
-------captial:30000000.000000
--------------2013-07-31 00:00:00total_earn:0.000000
date:2013-08-30 00:00:00
-------captial:30000000.000000
--------------2013-08-30 00:00:00total_earn:0.000000
date:2013-09-30 00:00:00
-------captial:30000000.000000
--------------2013-09-30 00:00:00total_earn:0.000000
date:2013-10-31 00:00:00
-------captial:30000000.000000
--------------2013-10-31 00:00:00total_earn:0.000000
date:2013-11-29 00:00:00
-------captial:30000000.000000
--------------2013-11-29 00:00:00total_earn:0.000000
date:2013-12-31 00:00:00
-------captial:30000000.000000
--------------2013-12-31 00:00:00total_earn:0.000000
date:2014-01-30 00:00:00
-------captial:30000000.000000
--------------2014-01-30 00:00:00total_earn:0.000000
date:2014-02-28 00:00:00
-------captial:30000000.000000
--------------2014-02-28 00:00:00total_earn:0.000000
date:2014-03-31 00:00:00
-------captial:30000000.000000
--------------2014-03-31 00:00:00total_earn:0.000000
date:2014-04-30 00: